## Getting claims of the found entities

In [ ]:
import json
from LODlit import wd

In [ ]:
# importing the search results
with open("terms_found_en.json","r") as jf:
    entities_en = json.load(jf)
with open("terms_found_nl.json","r") as jf:
    entities_nl = json.load(jf)

In [ ]:
# take Qs from the 'terms found' results
# putting all unique QIDs in a list

qids = []

for term, values in entities_en.items():
    for v in values:
        qids.append(v["QID"])
        
for term, values in entities_nl.items():
    for v in values:
        qids.append(v["QID"])
        
# taking only unique QIDs         
unique_qids = list(set(qids))

In [ ]:
user_agent = "Bot getting claims of entities (nesterov@cwi.nl)"
props = ['P31','P279']
path = '/Users/anesterov/wd/jan31/claims/'

In [ ]:
# all claims are saved in batches
wd.get_claims(unique_qids,props,user_agent,path)

In [ ]:
# Merging all claims in one file
dir_path = '/Users/anesterov/wd/jan31/claims'
claims_paths = [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path,f)) and f.endswith('.json')]

In [ ]:
all_claims = {}
for file in claims_paths:
    with open(file, "r") as jf:
        claims = json.load(jf)
        all_claims.update(claims)

In [ ]:
# saving all claims in one file
with open("/Users/anesterov/wd/jan31/claims/all_claims.json", "w") as f:
    json.dump(all_claims,f)

## Getting labels of the property values

In [ ]:
# Getting unique value IDs of P31 and P279 from all claims
values_ids = []
for properties in all_claims.values():
    values_ids.extend(properties['P31'].keys())
    values_ids.extend(properties['P279'].keys())

In [ ]:
unique_values_ids = list(set(values_ids))

In [ ]:
len(unique_values_ids)

In [ ]:
user_agent = "bot getting labels of entities (nesterov@cwi.nl)"
langs = ['en','nl']

In [ ]:
# Returns a dict: {"labels":dict with labels by lang,"failed":list of QIDs that were failed}
labels = wd.get_labels_by_q(unique_values_ids,langs,user_agent)

In [ ]:
# checking if there are failed entities
len(labels['failed'])

In [ ]:
# exporting labels
with open("/Users/anesterov/wd/jan31/labels_property_values.json", "w") as f:
    json.dump(labels['labels'],f)

## Merging claims with property values labels

In [ ]:
# it overwrites all_claims
for claims in all_claims.values():
    for k in claims['P31'].keys():
        claims['P31'][k] = labels['labels'].get(k)
    for k in claims['P279'].keys():
        claims['P279'][k] = labels['labels'].get(k)

In [ ]:
# saving all claims with labels (languge independent)
with open("/Users/anesterov/wd/jan31/all_claims_with_labels.json", "w") as f:
    json.dump(all_claims,f)